In [1]:
import sys
sys.path.insert(0, 'D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Classes')

import warnings
warnings.filterwarnings('ignore')

from models import ClassificationModel, EmbeddingsModel
from trainer import SiameseTrainerCombinedLoss, all_subjects
import torch

import wandb
!wandb login

wandb: Currently logged in as: svenk. Use `wandb login --relogin` to force relogin


In [2]:
subjects_test = [all_subjects[0]]
subjects_train = [sub for sub in all_subjects if sub not in subjects_test]

In [3]:
sweep_config = {
    'method': 'random',
    "early_terminate": {
          "type": "hyperband",
          "max_iter": 10,
          "s": 2
    },
    "metric": {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    "parameters": {
        "lr": {'max': 0.01, 'min': 0.0001},
        "weight_decay": {'max': 0.01, 'min': 0.0001},
        "dropout": {'max': 0.6, 'min': 0.0},
        "lambda_loss": {'max': 0.8, 'min': 0.2}
    }
}

In [4]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        trainer_hyperparameter = {
            "path": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects.pkl",
            "subjects_train": subjects_train,
            "subjects_test": subjects_test,
            "dataset_ignore_sample_subject": False,
            "wandb": True,
            "log": False,
            "filter": [],
            "batch_size": 256,
            "batch_size_test": 64,
            "margin": 0.04,
            "number_steps": 2000,
            "learning_rate": config.lr,
            "lambda_loss": config.lambda_loss,
            "weight_decay": config.weight_decay
        }

        model_embedding_hyperparameter = {
            "sigmoid": True,
            "dropout": config.dropout,
            "layers": [207, 512, 64]
        }

        model_classifier_hyperparameter = {
            "dropout": config.dropout,
            "layers": [64, 32]
        }

        classifier_model = ClassificationModel(model_classifier_hyperparameter)
        embedding_model = EmbeddingsModel(model_embedding_hyperparameter)
        _ = embedding_model.load_state_dict(torch.load("D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Results\models\embedding_models\model_abs_acc_89.40.pth"))
        #filter = lambda data: data["label"].isin(trainer_hyperparameter["filter"])
        trainer = SiameseTrainerCombinedLoss(trainer_hyperparameter, classifier_model, embedding_model, filter=None)

        trainer.trainloop(3)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="PainLevelShiftDetection_Combined_Loss_SiameseNetwork")

In [ ]:
wandb.agent(sweep_id, train, count=30)